In [46]:
from tabulate import tabulate

f = open("input.txt", "r")

grid = []
starting_position = [0, 0]
found_starting_row = False

for line in f:
    grid.append(list(line.replace('\n', '')))
    if not found_starting_row:
        if 'S' not in line:
            starting_position[0]+=1
        else:
            starting_position[1]=line.index('S')
            found_starting_row = True

# print(tabulate(grid, tablefmt="rounded_grid"))

height = len(grid)
width = len(grid[0])

print(starting_position)

offsets = {(0, 1), (0, -1), (1, 0), (-1, 0)}

def computeNbOfPlots(position, nb_of_steps):
    visited_plots = set()
    visited_plots.add(tuple(position))
    nb_of_plots = [1, 0]
    last_plots = set()
    last_plots.add(tuple(position))
    steps = 1
    while steps <= nb_of_steps:
        new_plots = set()
        for (i, j) in last_plots:
            for (offset_i, offset_j) in offsets:
                (i_n, j_n) = (i+offset_i, j+offset_j)
                if 0 <= i_n < height and 0 <= j_n < width:
                    if grid[i_n][j_n] in ['.', 'S'] and (i_n, j_n) not in visited_plots:
                        new_plots.add((i_n, j_n))
                        # Since we're on a grid, each new plot we visit can be visited every other next step
                        # so we just keep track of the plots that can be reached in an even/odd number of steps
                        nb_of_plots[steps % 2] += 1
                        visited_plots.add((i_n, j_n))
        last_plots = new_plots
        steps += 1
    return nb_of_plots[nb_of_steps%2]

print(computeNbOfPlots(starting_position, 64))



[65, 65]
3594


In [47]:
from tabulate import tabulate

f = open("input.txt", "r")

grid = []
starting_position = [0, 0]
found_starting_row = False

for line in f:
    grid.append(list(line.replace('\n', '')))
    if not found_starting_row:
        if 'S' not in line:
            starting_position[0]+=1
        else:
            starting_position[1]=line.index('S')
            found_starting_row = True

height = len(grid)
width = len(grid[0])

print(starting_position)

offsets = {(0, 1), (0, -1), (1, 0), (-1, 0)}

def computeNbOfPlots(position, nb_of_steps):
    visited_plots = set()
    visited_plots.add(tuple(position))
    nb_of_plots = [1, 0]
    last_plots = set()
    last_plots.add(tuple(position))
    steps = 1
    while steps <= nb_of_steps:
        new_plots = set()
        for (i, j) in last_plots:
            for (offset_i, offset_j) in offsets:
                (i_n, j_n) = (i+offset_i, j+offset_j)
                if 0 <= i_n < height and 0 <= j_n < width:
                    if grid[i_n][j_n] in ['.', 'S'] and (i_n, j_n) not in visited_plots:
                        new_plots.add((i_n, j_n))
                        nb_of_plots[steps % 2] += 1
                        visited_plots.add((i_n, j_n))
        last_plots = new_plots
        steps += 1
    return nb_of_plots[nb_of_steps%2]

# The infinite pattern forms a diamond with four quadrants.
# Notice that there is an empty cross in the input garden: there are no obstacles in the row and column of S
# That means we can go from one garden to the next in width/2 (65) steps, or from one S to the next in width (131) steps
# With the given number of steps, we can figure out how wide our diamond of reachable gardens actually is: 26501365-65/131 = 202300
# We also notice that after a certain number of steps, the nb of reachable plots within a garden oscillate between two values, one for each parity.
# So for almost all the gardens in our big diamond, the nb of reachable plots is one of these two values. We just need to figure out which parity
# the number of steps has when we enter the garden. Since gardens are separated by an odd number of steps, the parity alternates and we get a checkboard pattern.
# Given the symmetry of the problem, we can split our diamond in 4 quadrants to simplify.
even_max = computeNbOfPlots((0,0),300)
odd_max = computeNbOfPlots((0,0),301)
target_steps = 26501365
max_gardens_in_straight_line = int((target_steps - 65)/131)
nb_of_gardens_with_same_parity_as_center_per_quadrant = 0
nb_of_gardens_with_different_parity_than_center_per_quadrant = 0
# Counting the gardens in the checkboard. For each step we take, there is a new diagonal of gardens of a given parity to count.
for i in range(1, max_gardens_in_straight_line):
    if i % 2 == 0:
        nb_of_gardens_with_same_parity_as_center_per_quadrant+=i
    else:
        nb_of_gardens_with_different_parity_than_center_per_quadrant+=i
# Now we can count the number of reachable plots in all the gardens which reach the limit value for their parity
plots_reachable_from_full_gardens = odd_max*(4*nb_of_gardens_with_same_parity_as_center_per_quadrant+1) + even_max*4*nb_of_gardens_with_different_parity_than_center_per_quadrant

# Now it gets more complicated: on the edge of the diamond, there aren't enough steps remaining to reach the limite value for the gardens. So we need to figure out
# where we enter these gardens and how many steps we have left at this point
# There are three cases: 
# - the end of the horizontal and vertical lines of our diamond, which are reached in a straight line from the starting position
# in the middle of their top, bottow, left or right side. At this point, only 130 steps remain (one short of reaching the next garden, as expected)
# - the gardens on the diagonals between the end of the lines: these are reached faster because we can enter through a corner. At this point 65+130 = 195 steps remain.
# - the gardens on the sides of the end of the lines, perpendicular to that line: these are reached slower, through a corner as well. At this point: 64 steps remain. 
# (Once we enter the last garden on the line, we need 66 steps of the remaining 130 to reach this one)
# The cases become apparent if you draw a line between the middle of the outer sides of all 4 gardens at the end of the vertical/horizontal lines. They split
# the gardens they go through and from trapezes and triangles.
starting_positions = {"top": (0, 65), "bottom": (height-1, 65), "left": (65, 0), "right": (65, width-1), "top-left": (0, 0), "top-right": (0, width-1), "bottom-left": (height-1, 0), "bottom-right": (height-1, width-1)}
plots_reachable_for_partial_gardens_end = [computeNbOfPlots(starting_positions[pos], 130) for pos in {"top", "bottom", "left", "right"}]
plots_reachable_for_partial_gardens_trapeze = [computeNbOfPlots(starting_positions[pos], 195) for pos in {"top-left", "top-right", "bottom-left", "bottom-right"}]
plots_reachable_for_partial_gardens_triangle = [computeNbOfPlots(starting_positions[pos], 64) for pos in {"top-left", "top-right", "bottom-left", "bottom-right"}]
# Now to count how many we have of each case.
# Per quadrant, only one garden is at the end of the horizontal/vertical line
nb_of_gardens_end_per_quadrant = 1
# Per quadrant, there as many trapezes as there are gardens in the line from the center one, minus one which is at the end
nb_of_gardens_trapeze_per_quadrant = max_gardens_in_straight_line-1
# Per quadrant, there as many triangles as there are gardens in the line from the center one
nb_of_gardens_triangle_per_quadrant = max_gardens_in_straight_line
# Ready to compute the total. The symmetry allows us not to specify the values for each quadrant.
total = plots_reachable_from_full_gardens + \
        nb_of_gardens_end_per_quadrant*sum(plots_reachable_for_partial_gardens_end) + \
        nb_of_gardens_trapeze_per_quadrant*sum(plots_reachable_for_partial_gardens_trapeze) + \
        nb_of_gardens_triangle_per_quadrant*sum(plots_reachable_for_partial_gardens_triangle)
print(total)

[65, 65]
605247138198755
